In [12]:
from mip import *

In [13]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instanceBig1 = {
    "m" : 6,
    "n" : 9,
    "l" : [190, 185, 185, 190, 195, 185],
    "s" : [11, 11, 23, 16, 2, 1, 24, 14, 20],
    "distances" : [
        [0, 200, 119, 28, 180, 77, 145, 61, 123, 87],
        [200, 0, 81, 207, 38, 122, 55, 138, 76, 113],
        [119, 81, 0, 126, 69, 121, 26, 117, 91, 32],
        [28, 207, 126, 0, 187, 84, 152, 68, 130, 94],
        [170, 38, 79, 177, 0, 92, 58, 108, 46, 98],
        [77, 122, 121, 84, 102, 0, 100, 16, 46, 96],
        [145, 55, 26, 152, 58, 100, 0, 91, 70, 58],
        [61, 138, 113, 68, 118, 16, 91, 0, 62, 87],
        [123, 76, 91, 130, 56, 46, 70, 62, 0, 66],
        [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
    ]
}

In [14]:
instance = instanceBig1

In [15]:
m = instance["m"]
n = instance["n"]
l = instance["l"]
s = instance["s"]
distances = np.array(instance["distances"])
max_distance = np.max(distances, axis=1).sum()

# depot is n+1

model = Model()

# Variables
# x[i,k] = j if courier i at time j is in location j
x = model.add_var_tensor((m, n+2), name="x", var_type=INTEGER, lb = 0, ub = n+1)

# y[i] is the total distance travelled by courier i
y = model.add_var_tensor((m,), name="y", var_type=INTEGER, lb = 0, ub = max_distance)

# z[i, j1, j2] = 1 if courier i travels from j1 to j2
z = model.add_var_tensor((m, n+1, n+1), name="z", var_type=BINARY)

# t[i,j] = 1 if courier i takes package j
t = model.add_var_tensor((m, n), name="t", var_type=BINARY)

# Constraints
# Each courier must start at the depot
model += x[:,0] == n+1
# Each courier must end at the depot
model += x[:,n+1] == n+1

bigM = 100


# t[i,j] = 1 if for some k x[i,k]=j
for i in range(m):
    for j in range(n):
        model += t[i,j] <= xsum(x[i,k] == j for k in range(m))
        model += t[i,j] >= xsum(x[i,k] == j for k in range(m)) - (1 - t[i,j]) * bigM

# Each package must be taken by exactly one courier
for j in range(n):
    model += np.sum(t[:,j]) == 1

# Each courier must take at most l[i] weight
for i in range(m):
    model += np.sum(t[i,:] * s) <= l[i]



# z[i, j1, j2] = 1 iff (x[i,k]=j1 and x[i,k+1]=j2)
for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):
            for k in range(n+1):
                # model += z[i,j1,j2] <= (x[i,k] - j1) + (1 - x[i,k+1]) * bigM
                # model += z[i,j1,j2] >= x[i,k+1] - j2 + (1 - x[i,k]) * bigM
                model += z[i,j1,j2] <= np.where(x[i,k]==j1, 1, 0).sum()
                model += z[i,j1,j2] <= np.where(x[i,k+1]==j2, 1, 0).sum()
                model += z[i,j1,j2] >= np.where(x[i,k]==j1, 1, 0).sum() + np.where(x[i,k+1]==j2, 1, 0).sum() - 1

# y[i]
for i in range(m):
    model += np.sum(z[i,:,:] * distances) == y[i]


v = model.add_var(name="v", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += v >= y[i]

model.threads = -1


Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [16]:
model.objective = minimize(v)

model.optimize()

Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18147 rows, 727 columns and 19434 nonzeros
Model fingerprint: 0xe65dd3be
Variable types: 0 continuous, 727 integer (654 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 12012 rows and 30 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


<OptimizationStatus.INFEASIBLE: 1>

In [17]:
print([y[i].x for i in range(m)])


[None, None, None, None, None, None]


In [18]:
print(model.objective_value)


None
